In [241]:
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from timeit import default_timer as timer
from scipy.special import softmax



In [242]:
df = pd.read_csv('ESS8_data.csv')

In [243]:
X = df[['SD1', 'PO1', 'UN1', 'AC1', 'SC1',
       'ST1', 'CO1', 'TR1', 'HD1', 'AC2', 'SC2', 'ST2',
       'CO2', 'PO2', 'BE2', 'TR2', 'HD2']].iloc[range(100),:]
X = X.to_numpy().T
N, M = X.T.shape
K = 3

In [244]:
np.random.seed(42)

p = np.random.permutation(range(N))[:K]
Z = X[:,p]
# Z = np.random.randint(1, 6, (M,K)) 

#Q = Z.T @ Z
#Qt = torch.tensor(Q,requires_grad=False).float()

R = X.T @ X
Rt = torch.tensor(R,requires_grad=False).float()

In [245]:
# applies both for eq. 7 and 8
def error(a_i,Qt,qt):
    return (0.5 * a_i.T @ Qt @ a_i) - (qt.T @ a_i)

In [246]:
def applyConstrains(M):
    return softmax(M, axis = 0)



def furthestSum(X):
    idx = np.random.choice(range(0,N))
    x_j = X[:,idx]
    max_vals = [0]
    max_vals = max_vals * K

    b_j = np.empty((K,1))
    b_j_norm = np.empty((K,1))

    for i in range(N):
        val = 0
        for j in range(M):
            val += np.abs(X[j, i] - x_j[j])
        max_idxs = [j for j, v in enumerate(max_vals) if val > v]

        if len(max_idxs) > 0:
            max_vals[max_idxs[0]] = val
    
    b_j = b_j.astype(np.float64)
            
    return b_j


        
    
    
print(b_j)    


[1 1 6 1 1 1 1 6 1 1 1 6 1 1 3 1 3]


In [247]:
start = timer()
torch.manual_seed(42)

RSS_values = list()
A = np.zeros((K,N))#.tolist()
B = np.zeros((N,K))#.tolist()

# LOOP UNTIL RSS IS LOW
for n in range(2): #N
    Q = Z.T @ Z
    Qt = torch.tensor(Q,requires_grad=False).float()

    # LOOP THROUGH ENTIRE A
    for i in range(N):
        q = Z.T @ X[:,i]
        qt = torch.tensor(q,requires_grad=False).float()
        
        v = furthestSum(X)
        #a_i = torch.tensor(v, requires_grad=True).float()
        #a_i = torch.autograd.Variable(torch.rand(K, 1), requires_grad=True) # eller er det Kx1 ?
        
        stop_loss = 1e-6
        step_size = 0.00001 # stop_loss / 3.0

        err = error(a_i,Qt,qt)
        print('Loss before: %s' % (torch.norm( err, p=2)))

        # TRAINING LOOP
        for k in range(10000): # 100000
            Delta = error(a_i,Qt,qt) 
            L = torch.norm(Delta, p=2)
            L.backward()
            a_i.data -= step_size * a_i.grad.data # step
            a_i.grad.data.zero_()
            if k % 10000 == 0: print('Loss for a is %s at iteration %i' % (L, k))
            if abs(L) < stop_loss:
                print('It took %s iterations to achieve %s loss.' % (k, step_size))
                break
        
        A[:,i] = np.array(a_i.tolist()).flatten() 
        
        # print('Loss after: %s' % (torch.norm( error(a_i,Qt,qt) )))

    ### apply softmax here? ###
    
    A = applyConstrains(A)
    Z = X @ A.T @ np.linalg.inv(A@A.T)
    
    # LOOP THROUGH ENTIRE B
    for i in range(K): #K
        r = X.T @ Z[:,i]
        rt = torch.tensor(r,requires_grad=False).float()

        b_i = torch.autograd.Variable(torch.rand(N, 1), requires_grad=True) 

        stop_loss = 1e-6
        step_size = 0.00001 # stop_loss / 3.0

        err = error(b_i,Rt,rt)
        print('Loss before: %s' % (torch.norm( err, p=2)))

        # TRAINING LOOP
        for k in range(10000): # 100000
            Delta = error(b_i,Rt,rt)
            L = torch.norm(Delta, p=2)
            L.backward()
            b_i.data -= step_size * b_i.grad.data # step
            b_i.grad.data.zero_()
            if k % 10000 == 0: print('Loss for b is %s at iteration %i' % (L, k))
            if abs(L) < stop_loss:
                print('It took %s iterations to achieve %s loss.' % (k, step_size))
                break

        B[:,i] = np.array(b_i.tolist()).flatten() 
        #print('Loss after: %s' % (torch.norm( error(b_i,Rt,rt) )))    
    
    # apply softmax here
    B = applyConstrains(B)
    Z = X @ B
    
    Zt = torch.tensor(Z, requires_grad=False).float()
    At = torch.tensor(A, requires_grad=False).float()
    Xt = torch.tensor(X,requires_grad=False).float()
    print("RSS at n=%s" % n, torch.norm(Xt-Zt@At,p='fro'))
    RSS_values.append( torch.norm(Xt-Zt@At,p='fro'))
    
end = timer()

print("It took: {0} seconds to finish running".format(end - start))
print("The best RSS value was", min(RSS_values))

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x17 and 3x3)

### Time at different data sizes
### Here we run 1000 inner loops and 10 in the outer
#### For X = 3 x 17  
##### It took: 17.8 seconds


#### For X = 6 x 17  
##### It took: 28.08 seconds


#### For X = 12 x 17
#### It took: 48 seconds




In [248]:
def furthestSum(X):
    idx = np.random.choice(range(0,N))
    x_j = X[:,idx]
    max_vals = [0]
    max_vals = max_vals * K

    b_j = np.empty((K,1))
    b_j_norm = np.empty((K,1))

    for i in range(N):
        val = 0
        for j in range(M):
            val += np.abs(X[j, i] - x_j[j])
        max_idxs = [j for j, v in enumerate(max_vals) if val > v]

        if len(max_idxs) > 0:
            max_vals[max_idxs[0]] = val
            b_j = X[:, i]
            
    return b_j

In [250]:
# Choose a random point for initialization
idx = int(np.random.choice(range(0,N)))
x_j = X[:,idx]

j_news = list()
j_news.append(idx)
excluded = list()

# Loop over the K archetypes
for n in range(K):
    best_val = 0
    best_idx = 0
    # Loop over all unseen samples
    for i in range(N):
        val = 0
        # sum over each element for each point
        for a in range(len(j_news)):
            for j in range(M):
                val += np.abs(X[j,i] - X[j , j_news[n]])
        if val > best_val:
            best_val = val
            best_idx = i
    
    
    j_news.append(best_idx)
            
        
            

            
    

#for i in range(N):
#    val = 0
#    for i in range(M):
#        val+= np.abs(X[j,i] - x_j[j])
#    
#    if val > current_best:
#        current_best = val
#        best_indexes.append(i)
#print(best_indexes)
#b_j = np.array([best_indexes[-3],best_indexes[-2],best_indexes[-1]])



[34, 83, 0, 83]

In [127]:
Y = np.zeros((2,3))

In [182]:
Y[:,1] = np.array([1,1])

In [183]:
Y

array([[0., 1., 0.],
       [0., 1., 0.]])

In [41]:
A = torch.rand(2, 2, requires_grad=False)
b = torch.rand(2, 1,  requires_grad=False)

In [238]:
idx = int(np.random.choice(range(0,N)))
x_j = X[:,idx]

j_news = np.empty(K+1)
j_news = j_news.astype(np.int64)
j_news[0] = idx


best_val = 0
best_idx = 0
for i in range(N):
    val = 0
    # sum over each element for each point
    for j in range(M): 
        val += np.abs(X[j,i] - X[j , j_news[0]])
        if val > best_val:
            best_val = val
            best_idx = i
                    
    j_news[0] = best_idx
print(j_news)

[                  48    48303276873128664 -9223372036854775808
                    0]
